# Tema 16: El TAD de las colas de prioridad

[José A. Alonso](https://www.cs.us.es/~jalonso)  
[Departamento de Ciencias de la Computación e I.A.](https://www.cs.us.es)  
[Universidad de Sevilla](http://www.us.es)  
Sevilla, 12 de agosto de 2019

> __Notas:__ 
+ La versión interactiva de este tema se encuentra en [Binder](https://mybinder.org/v2/gh/jaalonso/Temas_interactivos_de_PF_con_Haskell/master?urlpath=lab/tree/temas/Tema-16.ipynb).
+ Se desactiva el [corrector estilo de Haskell](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint).

In [1]:
:opt no-lint

Especificación del TAD de las colas de prioridad
================================================
 
Signatura del TAD colas de prioridad
------------------------------------

**Descripción de las colas de prioridad**

+ Una *cola de prioridad* es una cola en la que cada elemento tiene asociada
  una prioridad. La operación de extracción siempre elige el elemento de menor
  prioridad.

+ Ejemplos:
    + La cola de las ciudades ordenadas por su distancia al destino
      final. 
    + Las colas de las tareas pendientes ordenadas por su fecha de
      terminación. 

**Signatura de las colas de prioridad**

+ Signatura:

```sesion
vacia,   :: Ord a => CPrioridad a 
inserta, :: Ord a => a -> CPrioridad a -> CPrioridad a 
primero, :: Ord a => CPrioridad a -> a
resto,   :: Ord a => CPrioridad a -> CPrioridad a
esVacia, :: Ord a => CPrioridad a -> Bool 
valida   :: Ord a => CPrioridad a -> Bool
```

+ Descripción de las operaciones:
    + `vacia` es la cola de prioridad vacía.
    + `(inserta x c)` añade el elemento `x` a la cola de prioridad `c`.
    + `(primero c)` es el primer elemento de la cola de prioridad `c`.
    + `(resto c)` es el resto de la cola de prioridad `c`.
    + `(esVacia c)` se verifica si la cola de prioridad `c` es vacía.
    + `(valida c)` se verifica si `c` es una cola de prioridad válida.

Propiedades del TAD de las colas de prioridad
---------------------------------------------

+ `inserta x (inserta y c) == inserta y (inserta x c)`

+ `primero (inserta x vacia) == x`

+ Si `x <= y`, entonces  
  `primero (inserta y (inserta x c)) == primero (inserta x c)`

+ `resto (inserta x vacia) == vacia`

+ Si `x <= y`, entonces  
  `resto (inserta y (inserta x c)) == inserta y (resto (inserta x c))`

+ `esVacia vacia`

+ `not (esVacia (inserta x c))`


Implementaciones del TAD de las colas de prioridad
==================================================

Las colas de prioridad como listas
----------------------------------

In [2]:
module ColaDePrioridadConListas
    (CPrioridad,
     vacia,   -- Ord a => CPrioridad a 
     inserta, -- Ord a => a -> CPrioridad a -> CPrioridad a 
     primero, -- Ord a => CPrioridad a -> a
     resto,   -- Ord a => CPrioridad a -> CPrioridad a
     esVacia, -- Ord a => CPrioridad a -> Bool 
     valida   -- Ord a => CPrioridad a -> Bool
    ) where

-- Colas de prioridad mediante listas.
newtype CPrioridad a = CP [a]
  deriving (Eq, Show)

-- Ejemplo de cola de prioridad
--    ghci> cp1
--    CP [1,2,3,7,9]
cp1 :: CPrioridad Int
cp1 = foldr inserta vacia [3,1,7,2,9]

-- (valida c) se verifica si c es una cola de prioridad válida. Por
-- ejemplo, 
--    valida (CP [1,3,5])  ==  True
--    valida (CP [1,5,3])  ==  False
valida :: Ord a => CPrioridad a -> Bool
valida (CP xs) = ordenada xs
  where ordenada (x:y:zs) = x <= y && ordenada (y:zs)
        ordenada _        = True

-- vacia es la cola de prioridad vacía. Por ejemplo,
--    vacia  ==  CP []
vacia :: Ord a => CPrioridad a 
vacia = CP []

-- (inserta x c) es la cola obtenida añadiendo el elemento x a la cola
-- de prioridad c. Por ejemplo,  
--    cp1            ==  CP [1,2,3,7,9]
--    inserta 5 cp1  ==  CP [1,2,3,5,7,9]
inserta :: Ord a => a -> CPrioridad a -> CPrioridad a 
inserta x (CP q) = CP (ins x q)
  where ins y []                   = [y]
        ins y r@(e:r') | y < e     = y:r
                       | otherwise = e:ins y r'

-- Nota. inserta usa O(n) pasos.

-- (primero c) es el primer elemento de la cola de prioridad c. Por
-- ejemplo, 
--    cp1          ==  CP [1,2,3,7,9]
--    primero cp1  ==  1
primero :: Ord a => CPrioridad a -> a
primero (CP(x:_)) = x
primero _         = error "primero: cola de prioridad vacia"

-- (resto c) es la cola de prioridad obtenida eliminando el primer
-- elemento de la cola de prioridad c. Por ejemplo,  
--    cp1        ==  CP [1,2,3,7,9]
--    resto cp1  ==  CP [2,3,7,9]
resto :: Ord a => CPrioridad a -> CPrioridad a
resto (CP (_:xs)) = CP xs
resto _           = error "resto: cola de prioridad vacia"

-- Nota. resto usa O(1) pasos.

-- (esVacia c) se verifica si la cola de prioridad c es vacía. Por
-- ejemplo,   
--    esVacia cp1    ==  False
--    esVacia vacia  ==  True
esVacia :: Ord a => CPrioridad a -> Bool 
esVacia (CP xs) = null xs

+ Ejemplos

In [3]:
cp1 = foldr inserta vacia [3,1,7,2,9]

In [4]:
cp1

CP [1,2,3,7,9]

In [5]:
inserta 5 cp1

CP [1,2,3,5,7,9]

In [6]:
primero cp1

1

In [7]:
resto cp1

CP [2,3,7,9]

In [8]:
esVacia cp1

False

In [9]:
esVacia vacia

True

+ Borra la primera implementación

Las colas de prioridad como montículos
--------------------------------------

La implementación de las colas de prioridad como montículos se estudiará en en el
[tema 20](tema-20.ipynb) (El TAD de los montículos).

Comprobación de las propiedades con QuickCheck
==============================================

In [10]:
module ColaDePrioridadPropiedades where

import ColaDePrioridadConListas
import Test.QuickCheck

-- ---------------------------------------------------------------------
-- Generador de colas de prioridad
-- ---------------------------------------------------------------------

-- genCPrioridad es un generador de colas de prioridad. Por ejemplo,
--    ghci> sample genCPrioridad
--    CP []
--    CP []
--    CP [-4]
--    CP [-2,-1,-1,2,5]
--    CP [-8,-5,4,6,8]
--    CP [-4,-1,3,3,6,7,10,10,10,10]
--    CP [-12,-10,-9,-7,2,4,6,7,7,9]
--    CP [-14,-12,-12,-7,-7,-4,4,9,14]
--    CP [-10,-9,-5,14]
--    CP [18]
--    CP [-19,-17,-16,-15,-13,-13,-13,-12,-6,-5,-3,0,2,3,4,5,8,18]
genCPrioridad :: (Arbitrary a, Num a, Ord a) =>  Gen (CPrioridad a)
genCPrioridad = do
  xs <- listOf arbitrary
  return (foldr inserta vacia xs)

-- La colas de prioridad son una concreción de la clase arbitraria. 
instance (Arbitrary a, Num a, Ord a) => Arbitrary (CPrioridad a) where
  arbitrary = genCPrioridad

-- Prop.: Las colas de prioridad producidas por genCPrioridad son
-- válidas. 
prop_genCPrioridad_correcto ::  CPrioridad Int -> Bool
prop_genCPrioridad_correcto c = valida c

-- Comprobación.
--    ghci> quickCheck prop_genCPrioridad_correcto
--    +++ OK, passed 100 tests.

-- ---------------------------------------------------------------------
-- Propiedades
-- ---------------------------------------------------------------------

-- Propiedad. Si se añade dos elementos a una cola de prioridad se
-- obtiene la misma cola de prioridad idependientemente del orden en
-- que se añadan los elementos.
prop_inserta_conmuta :: Int -> Int -> CPrioridad Int -> Bool
prop_inserta_conmuta x y c =
  inserta x (inserta y c) == inserta y (inserta x c)

-- Comprobación.
--    ghci> quickCheck prop_inserta_conmuta
--    +++ OK, passed 100 tests.

-- Propiedad. La cabeza de la cola de prioridad obtenida anadiendo un
-- elemento x a la cola de prioridad vacía es x.
prop_primero_inserta_vacia :: Int -> CPrioridad Int -> Bool
prop_primero_inserta_vacia x _ =
  primero (inserta x vacia) == x

-- Comprobación.
--    ghci> quickCheck prop_primero_inserta_vacia
--    +++ OK, passed 100 tests.
 
-- Propiedad. El primer elemento de una cola de prioridad c no cambia
-- cuando se le añade un elemento mayor o igual que algún elemento de c. 
prop_primero_inserta :: Int -> Int -> CPrioridad Int -> Property
prop_primero_inserta x y c =
  x <= y ==> primero (inserta y c') == primero c'
  where c' = inserta x c

-- Comprobación.
--    ghci> quickCheck prop_primero_inserta
--    +++ OK, passed 100 tests.

-- Propiedad. El resto de añadir un elemento a la cola de prioridad
-- vacía es la cola vacía. 
prop_resto_inserta_vacia :: Int -> Bool
prop_resto_inserta_vacia x =
  resto (inserta x vacia) == vacia

-- Comprobación.
--    ghci> quickCheck prop_resto_inserta_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. El resto de la cola de prioridad obtenida añadiendo un
-- elemento y a una cola c' (que tiene algún elemento menor o igual que
-- y) es la cola que se obtiene añadiendo y al resto de c'.
prop_resto_inserta :: Int -> Int -> CPrioridad Int -> Property
prop_resto_inserta x y c =
  x <= y ==> resto (inserta y c') == inserta y (resto c')
  where c' = inserta x c

-- Comprobación:
--    ghci> quickCheck prop_resto_inserta
--    +++ OK, passed 100 tests.

-- Propiedad. vacia es una cola vacía.
prop_vacia_es_vacia :: Bool
prop_vacia_es_vacia = esVacia (vacia :: CPrioridad Int)

-- Comprobación.
--    ghci> quickCheck prop_vacia_es_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. Si se añade un elemento a una cola de prioridad se obtiene
-- una cola no vacía.
prop_inserta_no_es_vacia :: Int -> CPrioridad Int -> Bool
prop_inserta_no_es_vacia x c =
  not (esVacia (inserta x c))

-- Comprobación.
--    ghci> quickCheck prop_inserta_no_es_vacia
--    +++ OK, passed 100 tests.

Comprobación de las propiedades
-------------------------------

In [11]:
import Test.QuickCheck
quickCheck prop_inserta_conmuta
quickCheck prop_primero_inserta_vacia
quickCheck prop_primero_inserta
quickCheck prop_resto_inserta_vacia
quickCheck prop_resto_inserta
quickCheck prop_vacia_es_vacia
quickCheck prop_inserta_no_es_vacia

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests; 61 discarded.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests; 95 discarded.

+++ OK, passed 1 test.

+++ OK, passed 100 tests.

> **Nota** Se borran los ficheros de los módulos usados

In [12]:
:! rm -f *.hs *.hi *.o *.dyn_*

Referencias
===========

+ F. Rabhi y G. Lapalme
  [Algorithms: A functional programming approach](http://bit.ly/1mWBqiI)
    + Cap. 5.4. Priority queues.